# Test Auto Generate Annotations

这个notebook用于测试 auto_generate_annotations 方法。该方法位于 `src/williamtoolbox/annotation.py`。

## 方法说明

auto_generate_annotations 方法用于自动对文档生成批注。它需要以下参数：
- rag_name: RAG服务的名称
- doc: 文档内容

In [ ]:
import asyncio
from williamtoolbox.annotation import auto_generate_annotations
import os

# 设置测试用的文档内容
test_doc = """
机器学习是人工智能的一个分支，它主要研究如何使用计算机模拟或实现人类的学习行为。
机器学习算法可以根据数据自动学习规律，并对新数据进行预测。

深度学习是机器学习的一个重要方向，它通过构建具有多层处理层的人工神经网络，
可以学习数据中的抽象特征表示。深度学习在图像识别、语音识别等领域取得了突破性进展。

强化学习是另一个重要的机器学习方法，它通过agent与环境的交互来学习最优策略。
在游戏、机器人控制等领域有广泛应用。
"""

## 准备RAG服务

在测试之前，我们需要确保有一个可用的RAG服务。这里我们使用命令行启动一个RAG服务：

In [ ]:
# 确保我们在正确的工作目录
!pwd

# 启动RAG服务
!byzerllm deploy --pretrained_model_type custom/bge \
--cpus_per_worker 0.001 \
--gpus_per_worker 0 \
--worker_concurrency 10 \
--model_path ~/.auto-coder/storage/models/AI-ModelScope/bge-large-zh \
--infer_backend transformers \
--num_workers 1 \
--model emb

# 启动一个RAG服务示例
!auto-coder.rag serve \
--model deepseek_chat \
--tokenizer_path ~/.auto-coder/storage/models/deepseek-ai/deepseek-llm-67b-chat \
--doc_dir /tmp/test-docs \
--port 8001 \
--host 0.0.0.0

## 测试 auto_generate_annotations

In [ ]:
async def test_auto_generate():
    try:
        # 调用方法生成批注
        result = await auto_generate_annotations("test_rag", test_doc)
        
        # 打印结果
        print("文档内容:")
        print(result.doc_text)
        print("\n生成的批注:")
        for i, annotation in enumerate(result.annotations, 1):
            print(f"\n批注 {i}:")
            print(f"文本: {annotation.text}")
            print(f"批注: {annotation.comment}")
            
    except Exception as e:
        print(f"测试失败: {str(e)}")

# 运行测试
await test_auto_generate()

## 测试多个文档场景

In [ ]:
# 测试具有多个段落和不同主题的文档
test_doc_2 = """
自然语言处理（NLP）是人工智能中一个重要的研究方向。
它致力于让计算机理解和处理人类语言。近年来，基于Transformer的模型
如BERT、GPT等在多个NLP任务上取得了显著进展。

计算机视觉是另一个重要领域，主要研究如何让计算机理解图像和视频内容。
卷积神经网络（CNN）是计算机视觉中最常用的深度学习模型之一。
"""

async def test_multiple_docs():
    try:
        # 对多个文档生成批注
        results = await asyncio.gather(
            auto_generate_annotations("test_rag", test_doc),
            auto_generate_annotations("test_rag", test_doc_2)
        )
        
        # 打印结果
        for i, result in enumerate(results, 1):
            print(f"\n文档 {i} 的批注:")
            for j, annotation in enumerate(result.annotations, 1):
                print(f"\n批注 {j}:")
                print(f"文本: {annotation.text}")
                print(f"批注: {annotation.comment}")
                
    except Exception as e:
        print(f"测试失败: {str(e)}")

# 运行多文档测试
await test_multiple_docs()

## 测试错误处理

In [ ]:
async def test_error_cases():
    try:
        # 测试空文档
        print("测试空文档:")
        result = await auto_generate_annotations("test_rag", "")
        print("结果:", result)
    except Exception as e:
        print(f"预期的错误: {str(e)}")
        
    try:
        # 测试不存在的RAG服务
        print("\n测试不存在的RAG服务:")
        result = await auto_generate_annotations("non_existent_rag", test_doc)
        print("结果:", result)
    except Exception as e:
        print(f"预期的错误: {str(e)}")

# 运行错误测试
await test_error_cases()

## 测试 chat_with_rag

接下来我们要测试 chat_with_rag 方法。这个方法用于与指定的 RAG 服务进行聊天交互。我们将使用与 auto_generate_annotations 中相似的 RAG 调用方式。

In [ ]:
from williamtoolbox.annotation import chat_with_rag

async def test_chat_with_rag():
    try:
        # 准备测试消息
        messages = [
            {"role": "user", "content": "你能告诉我机器学习是什么吗？"}
        ]
        
        # 调用 chat_with_rag 方法
        response = await chat_with_rag("test_rag", messages)
        
        print("RAG 回答:")
        print(response)
        
    except Exception as e:
        print(f"测试失败: {str(e)}")

# 运行测试
await test_chat_with_rag()

In [ ]:
async def test_chat_with_rag_conversation():
    try:
        # 准备多轮对话消息
        messages = [
            {"role": "user", "content": "什么是深度学习？"},
            {"role": "assistant", "content": "深度学习是机器学习的一个分支，主要使用多层神经网络进行学习。"},
            {"role": "user", "content": "那神经网络是什么？"}
        ]
        
        # 调用 chat_with_rag 方法进行多轮对话
        response = await chat_with_rag("批注知识库", messages)
        
        print("多轮对话测试:")
        print("问题历史:")
        for msg in messages:
            print(f"{msg['role']}: {msg['content']}")
        print("\nRAG 回答:")
        print(response)
        
    except Exception as e:
        print(f"测试失败: {str(e)}")

# 运行多轮对话测试
await test_chat_with_rag_conversation()

In [ ]:
async def test_chat_with_rag_errors():
    try:
        # 测试空消息列表
        print("测试空消息列表:")
        result = await chat_with_rag("test_rag", [])
        print("结果:", result)
    except Exception as e:
        print(f"预期的错误: {str(e)}")
        
    try:
        # 测试不存在的RAG服务
        print("\n测试不存在的RAG服务:")
        messages = [{"role": "user", "content": "测试消息"}]
        result = await chat_with_rag("non_existent_rag", messages)
        print("结果:", result)
    except Exception as e:
        print(f"预期的错误: {str(e)}")

# 运行错误测试
await test_chat_with_rag_errors()